# Прием и обработка твитов микробатчем

## Инициализация

In [1]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.toree.kernel.api
import java.util.Calendar

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


## Чтение модели

In [3]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)

modelPath = /home/jovyan/models/spark-ml-model
model = pipeline_6cb85880407d


pipeline_6cb85880407d

## Определяем схему и инициируем потоковый датафрейм

In [4]:
val inputStreamPath = "/home/jovyan/work/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("arrived_key", StringType)
    .add("timestamp", TimestampType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/work/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(arrived_key,StringType,true), StructField(timestamp,TimestampType,true))
inputDF = [tweet: string, hiddentargetclue: int ... 2 more fields]


[tweet: string, hiddentargetclue: int ... 2 more fields]

## Глобальная переменная для удобства просмотра датасета в отдельной ячейке

In [5]:
var globDF:DataFrame = null

globDF: org.apache.spark.sql.DataFrame = null


## Внимание!
- К сожалению, я не нашел внятной доки по апи ядра toree, чтобы нормально выводить оперативно изменяющийся датасет в этом ноутбуке
- В связи с этим см. след. пункт
- В блоке приема твитов ниже вывод датасета для удобства просмотра осуществляется через глобальную переменную globDF, которая просматривается при помощи выполнения блока, следующего за приемом твитов

## Микробатч приема твитов

In [6]:
// Определяем udf для получения probability по 0 и 1
val getProbability =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector, pos: Integer) =>
        {
            prediction(pos)
        }
    )

var fRuns = 0

// Микробатч для вывода результата предсказания
// Выводится вероятность негативного твита
// В задании написано, что это последняя колонка, но она здесь вроде первая (в позиции 0)
val stream = inputDF.writeStream.foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        try {
            fRuns += 1
            print(s"${Calendar.getInstance().toInstant} - loaded from the events stream $fRuns times"+13.toChar)
            // Применяем модель и получаем соотв. датасет с предсказаниями
            globDF = 
                model.transform(batchDF)
                    .select(
                        $"arrived_key",
                        $"timestamp",
                        $"tweet",
                        // $"hiddentargetclue",
                        (getProbability($"probability",lit(0))).alias("Negative Probability")
                    )
        } catch {
            case e:Throwable => {
                print(e.getMessage.replaceAll("\n"," "))
                print(13.toChar)
            }
        }
    }
}.start()

getProbability = UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))
fRuns = 0
stream = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@35a036eb


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@35a036eb

## Блок просмотра результата
- Этот блок предназначен для просмотра результата потоковой обработки твитов и применения модели из предыдущего блока
- Каждый раз при выполнении этого блока будет выводиться оперативное состояние датасета из блока выше

In [14]:
%%dataframe --limit=100
globDF

arrived_key,timestamp,tweet,Negative Probability
2020-01-28 07:49:32.801 - 46198,2020-01-28 09:49:32.801,"aww man... @nomtweets sounds delicious, but there's nothing tasty about them at all. boo-urns",0.5120172035845034
2020-01-28 07:49:32.801 - 13443,2020-01-28 09:49:32.801,@mrscurvy I love to interact with people and no body's here. *tears* except the staff and it's a skeleton crew today.,0.48005796380371935
2020-01-28 07:49:32.801 - 15546,2020-01-28 09:49:32.801,"MY HUBBY @EmoneyAKAweezy WANTS ME TO HIT IMPERIAL TONIGHT, BUT: I GOTTA GO OT WITH MAX UGGGH. I MISS MY BEBE. OWWW",0.6556136403895654
2020-01-28 07:49:32.801 - 22021,2020-01-28 09:49:32.801,@curexcomplex sure is!,0.4884236449783609
2020-01-28 07:49:32.801 - 43841,2020-01-28 09:49:32.801,"@FizzyDuck i know, it's just the last time, all that came up was a borked link. but now i shall go and snoop about UFOs!",0.5127155838348411
2020-01-28 07:49:32.801 - 45891,2020-01-28 09:49:32.801,I really dont feel like washing my Jeep this week,0.5761298796781984
2020-01-28 07:49:32.801 - 55695,2020-01-28 09:49:32.801,@dmeeno Woke up and couldn't get back to sleep,0.539775721703144
2020-01-28 07:49:32.801 - 66650,2020-01-28 09:49:32.801,@emilybourne except she got bored and went outside,0.4979333211092284
2020-01-28 07:49:32.801 - 71033,2020-01-28 09:49:32.801,"is hanging with chey tonight, hurry and get off work",0.5231740833608696
2020-01-28 07:49:32.801 - 75281,2020-01-28 09:49:32.801,I have officially become #jealousengage09,0.4884236449783609


## Останов чтения потока

In [15]:
globDF = null
stream.stop()

globDF: org.apache.spark.sql.DataFrame = null
